### 🧠 What is Query Decomposition?
Query decomposition is the process of taking a complex, multi-part question and breaking it into simpler, atomic sub-questions that can each be retrieved and answered individually.

#### ✅ Why Use Query Decomposition?

- Complex queries often involve multiple concepts

- LLMs or retrievers may miss parts of the original question

- It enables multi-hop reasoning (answering in steps)

- Allows parallelism (especially in multi-agent frameworks)

In [1]:
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnableSequence

c:\MY_Folder\Complete_RAG_Course\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Step 1: Load and embed the document
loader = TextLoader("langchain_crewai_dataset.txt")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding)
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 4, "lambda_mult": 0.7})

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

llm = init_chat_model(model="groq:llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000224166B6E90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000224166B7890>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
# Step 3: Query decomposition
decomposition_prompt = PromptTemplate.from_template("""
You are an AI assistant. Decompose the following complex question into 2 to 4 smaller sub-questions for better document retrieval.

Question: "{question}"

Sub-questions:
""")
decomposition_chain = decomposition_prompt | llm | StrOutputParser()

In [ ]:
query = "How does LangChain use memory and agents compared to CrewAI?"
decomposition_question = decomposition_chain.invoke({"question": query})


In [8]:
print(decomposition_question)

To better understand the context and retrieve relevant information, I'll break down the complex question into 4 smaller sub-questions:

1. **What is LangChain and what are its core features?**
   This sub-question aims to understand the fundamental components and functionality of LangChain, allowing for a comparison with CrewAI.

2. **How does LangChain utilize memory in its architecture?**
   This sub-question delves into the specifics of LangChain's memory management, enabling a comparison with CrewAI's approach.

3. **What are the key components and features of CrewAI?**
   This sub-question focuses on understanding the core features and components of CrewAI, which will facilitate a meaningful comparison with LangChain.

4. **How do LangChain and CrewAI differ in their use of agents or conversational interfaces?**
   This sub-question explores the differences in agent-based architecture and conversational interfaces between LangChain and CrewAI, providing valuable insights into thei

In [9]:
# Step 4: QA chain per sub-question
qa_prompt = PromptTemplate.from_template("""
Use the context below to answer the question.

Context:
{context}

Question: {input}
""")
qa_chain = create_stuff_documents_chain(llm=llm, prompt=qa_prompt)

In [10]:
# Step 5: Full RAG pipeline logic
def full_query_decomposition_rag_pipeline(user_query):
    # Decompose the query
    sub_qs_text = decomposition_chain.invoke({"question": user_query})
    sub_questions = [q.strip("-•1234567890. ").strip() for q in sub_qs_text.split("\n") if q.strip()]
    
    results = []
    for subq in sub_questions:
        docs = retriever.invoke(subq)
        result = qa_chain.invoke({"input": subq, "context": docs})
        results.append(f"Q: {subq}\nA: {result}")
    
    return "\n\n".join(results)

In [11]:
# Step 6: Run
query = "How does LangChain use memory and agents compared to CrewAI?"
final_answer = full_query_decomposition_rag_pipeline(query)
print("✅ Final Answer:\n")
print(final_answer)

✅ Final Answer:

Q: To better understand the comparison between LangChain and CrewAI, I would decompose the complex question into the following 4 smaller sub-questions:
A: It seems you've started your sub-questions. Here are the four smaller sub-questions to help compare LangChain and CrewAI, based on the context provided:

1. How do LangChain agents operate?
2. What tasks does LangChain handle in a hybrid system?
3. What tasks does CrewAI manage in a hybrid system?
4. How do CrewAI and LangChain collaborate in a hybrid system?

Q: **What is the memory architecture used in LangChain?**
A: The memory architecture used in LangChain includes modules like ConversationBufferMemory and ConversationSummaryMemory. These modules allow the LLM to maintain awareness of previous conversation turns or summarize long interactions to fit within token limits.

Q: This sub-question aims to understand the structure and functionality of LangChain's memory, which will help in comparing it with CrewAI's me